In [1]:
from bs4 import BeautifulSoup     #网页解析，获取数据\n",
import re       #正则表达式，进行文字匹配\n",
import urllib.request,urllib.error      #制定URL，获取网页数据\n",
# import xlwt     #进行excel操作\n",
import sqlite3  #进行SQLite数据库操作\n",
import pandas as pd #存储数据及处理数据\n",
import numpy as np
import requests
import random
import time
from collections import defaultdict
from collections import deque
from collections import Counter
import collections
import json
import sys
import datetime
import os
import csv
from math import *
import pymysql
# from geopy.distance import geodesic
# from pyproj import Proj
pd.set_option("display.max_columns", 100)
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
# 阈值可以设置成字典，对每一步的阈值进行设置
# threshold_dict={1:2,2:4}
# threshold_dict[2]
def simulate(code_ls,df_net_new,threshold,ls_dese):
    _district_industryCOs=code_ls
    _simulate_list = []
    _simulate_list.append(_district_industryCOs)
    # 从一步开始向多步扩展
    flag = 1
    t = 1
    ls_forcast = []
    while flag == 1:
        _exist_list = []
        for k in range(0,len(_simulate_list)):
            _exist_list.extend(_simulate_list[k])
        c = 0
        _possible_list = []
        for j in range(len(_simulate_list[len(_simulate_list)-1])):
            _industryCO = _simulate_list[len(_simulate_list)-1][j]
            #if _industryCO in industryCOs_product:
            # 从每一个优势产业开始向外找，一步联系值超过的节点
            _df_industryCO_proximity = df_net_new.loc[((df_net_new['Source']==_industryCO) | (df_net_new['Target']==_industryCO))&(df_net_new['Weight']>=threshold)]
            if len(_df_industryCO_proximity)>0:
                _industryCOs_1 = _df_industryCO_proximity.loc[:,'Source'].unique()
                _industryCOs_2 = _df_industryCO_proximity.loc[:,'Target'].unique()

                for m in range(len(_industryCOs_1)):
                    _industryCO_ = _industryCOs_1[m]
                    if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) and (_industryCO_ not in ls_dese):
                        _possible_list.append(_industryCO_)
                        ls_a=[]
                        ls_a.append(t)
                        ls_a.append(_industryCO)
                        ls_a.append(_industryCO_)
                        ls_a.append('上游')
                        ls_forcast.append(ls_a)
                        #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                        #f.write(ss)
                        c= c+1

                for m in range(len(_industryCOs_2)):
                    _industryCO_ = _industryCOs_2[m]
                    if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) and (_industryCO_ not in ls_dese):
                        _possible_list.append(_industryCO_)
                        ls_a=[]
                        ls_a.append(t)
                        ls_a.append(_industryCO)
                        ls_a.append(_industryCO_)
                        ls_a.append('下游')
                        ls_forcast.append(ls_a)
                        #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                        #f.write(ss)
                        c= c+1
        if c == 0:
            flag=0
        else:
            flag = 1
            t = t + 1
            _possible_list = list(set(_possible_list))
            _simulate_list.append(_possible_list)
    return ls_forcast

# 

### 图谱切割生成

In [3]:
# 根据模拟节点得到这些节点的邻接矩阵
# 返回的邻接矩阵是一个列表，可以用np.array()来转变为矩阵
def adjacence_matrix(code_ls, dataframe):
    # code_ls是产业链已有产品节点编码和预测出的节点编码一起组成的节点编码表
    # dataframe是节点间联系表
    df_temp = dataframe.loc[(dataframe['Source'].isin(code_ls)) & (dataframe['Target'].isin(code_ls))]

    # 根据code_ls生成各节点编码的标识字典
    member_dict = {}
    name_ = 0
    for i in code_ls:
        member_dict[i] = name_
        name_ += 1

    # 节点个数为len(code_ls)，基础矩阵为g_
    n = len(code_ls)
    g_ = [[0 for i in range(n)] for i in range(n)]

    # 根据节点间的联系数据将邻接矩阵填入数据
    df_temp_1 = df_temp.loc[:, ['Source', 'Target']]
    df_temp_1_ls = df_temp_1.values.tolist()
    for i in df_temp_1_ls:
        x_index = member_dict[i[0]]
        y_index = member_dict[i[1]]

        g_[x_index][y_index] = 1
        g_[y_index][x_index] = 1

    return g_, member_dict


# 根据邻接矩阵求得可达矩阵
def get_reach_matrix(matrix):
    # matrix是邻接矩阵

    # 输入邻接矩阵和相同长度的单位矩阵
    adj_matrix = np.mat(matrix)
    identity_matrix = np.identity(len(adj_matrix))
    # 第k+1步更新的矩阵
    new_matrix_ = adj_matrix + identity_matrix
    # 第k步更新的矩阵
    old_matrix_ = new_matrix_

    flag = 0
    step = 1
    while flag == 0:
        old_matrix_ = new_matrix_
        new_matrix_ = old_matrix_ * (adj_matrix + identity_matrix)
        for i in range(len(new_matrix_)):
            for j in range(len(new_matrix_)):
                if new_matrix_[i, j] >= 1:
                    new_matrix_[i, j] = 1
        step += 1
        if (old_matrix_ == new_matrix_).all():
            flag = 1

    return new_matrix_


# 最短路径法
# 广度优先搜索，一层一层往下搜索，直到找到终点为止，耗时有点长
def find_shortest_path_(start_node, end_node, all_nodes, ls_forbidden, df_net):
    # start_node是起点产品节点的编码，str
    # end_node是终点产品节点的编码，str
    # all_nodes是图谱里的所有产品节点，包含起点和终点，list
    # ls_forbidden是设定好的排除产品，不会出现在模拟结果中
    # df_net是全国产品节点间联系数据，dataframe

    # 初始化起点集合
    s_1 = [[start_node]]
    s_2 = [start_node]
    # 初始化剩余点集合
    u = [x for x in all_nodes if x not in s_2]
    t = 1
    ls_forecast = []

    while end_node in u:
        ls_nodes_temp_ = []
        for i in s_1[-1]:
            df_temp_ = df_net.loc[(df_net['primary_code'] == i) | (df_net['related_code'] == i)]
            if len(df_temp_) > 0:
                ls_temp_ = df_temp_.loc[:, ['primary_code', 'related_code']].values.tolist()
                ls_temp_ = [a for x in ls_temp_ for a in x]
                ls_temp_ = [x for x in ls_temp_ if x not in s_2 and x not in ls_forbidden]
                ls_temp_ = list(Counter(ls_temp_))

                for j in ls_temp_:
                    ls_forecast_i = []
                    df_weight = df_temp_.loc[(df_temp_['primary_code'] == j) | (df_temp_['related_code'] == j)]
                    weight = np.mean(df_weight['primary_weight'].tolist())
                    ls_forecast_i.extend([t, i, j, weight])
                    ls_forecast.append(ls_forecast_i)
                ls_nodes_temp_.extend(ls_temp_)
            else:
                continue
        ls_nodes_temp_ = list(Counter(ls_nodes_temp_))
        if len(ls_nodes_temp_) == 0:
            return [], []
        else:
            s_1.append(ls_nodes_temp_)
            s_2.extend(ls_nodes_temp_)
            t += 1
            u = [x for x in all_nodes if x not in s_2]
            # print(t, ls_nodes_temp_)
    # df_forecast 是最少步数可以从起点走到终点的数据集合
    df_forecast = pd.DataFrame(ls_forecast, columns=['t', 'origin', 'destination', 'weight'])
    location_end = np.where(df_forecast == end_node)
    if len(location_end[0]) != 0:
        # 根据路径数据，还原从起点到终点的路径path
        max_t = max(df_forecast['t'].tolist())
        t_temp = max_t
        path = [[end_node]]
        while t_temp >= 1:
            path_a = []
            for i in path:
                df_temp = df_forecast.loc[(df_forecast['t'] == t_temp) & (df_forecast['destination'] == i[0])]
                df_temp_ls = df_temp['origin'].tolist()

                for a in df_temp_ls:
                    ls_a = [a]
                    ls_a.extend(i)
                    path_a.append(ls_a)

            path = path_a
            t_temp -= 1

        # 根据path列表求得每条路径对应的代价之和
        path_weight = []
        for i in path:
            weight_i = []
            for j in range(len(i) - 1):
                df_weight = df_forecast.loc[(df_forecast['origin'] == i[j]) & (df_forecast['destination'] == i[j + 1])]
                weight_ij = df_weight['weight'].tolist()[0]
                weight_i.append(1 / weight_ij)
            path_weight.append(sum(weight_i))
        # 根据path_weight列表找到代价最小的那条路径
        path_weight_min = min(path_weight)
        path_min = [path[x] for x in range(len(path)) if path_weight[x] == path_weight_min]
        return path_min, path_weight_min
    else:
        return [], []

In [4]:
# 最短路径模拟
def find_shortest_path_(start_node, end_node, all_nodes, ls_forbidden, df_net):
    # 初始化起点集合
    s_1 = [[start_node]]
    s_2 = [start_node]
    # 初始化剩余点集合
    u = [x for x in all_nodes if x not in s_2]
    t = 1
    ls_forecast = []

    while end_node in u:
        ls_nodes_temp_ = []
        for i in s_1[-1]:
            df_temp_ = df_net.loc[(df_net['primary_code'] == i) | (df_net['related_code'] == i)]
            if len(df_temp_) > 0:
                ls_temp_ = df_temp_.loc[:, ['primary_code', 'related_code']].values.tolist()
                ls_temp_ = [a for x in ls_temp_ for a in x]
                ls_temp_ = [x for x in ls_temp_ if x not in s_2 and x not in ls_forbidden]
                ls_temp_ = list(Counter(ls_temp_))

                for j in ls_temp_:
                    ls_forecast_i = []
                    df_weight = df_temp_.loc[(df_temp_['primary_code'] == j) | (df_temp_['related_code'] == j)]
                    weight = np.mean(df_weight['primary_weight'].tolist())
                    ls_forecast_i.extend([t, i, j, weight])
                    ls_forecast.append(ls_forecast_i)
                ls_nodes_temp_.extend(ls_temp_)
            else:
                continue
        ls_nodes_temp_ = list(Counter(ls_nodes_temp_))
        if len(ls_nodes_temp_) == 0:
            return [], []
        else:
            s_1.append(ls_nodes_temp_)
            s_2.extend(ls_nodes_temp_)
            t += 1
            u = [x for x in all_nodes if x not in s_2]
            # print(t, ls_nodes_temp_)
    # df_forecast 是最少步数可以从起点走到终点的数据集合
    df_forecast = pd.DataFrame(ls_forecast, columns=['t', 'origin', 'destination', 'weight'])
    location_end = np.where(df_forecast == end_node)
    if len(location_end[0]) != 0:
        # 根据路径数据，还原从起点到终点的路径path
        max_t = max(df_forecast['t'].tolist())
        t_temp = max_t
        path = [[end_node]]
        while t_temp >= 1:
            path_a = []
            for i in path:
                df_temp = df_forecast.loc[(df_forecast['t'] == t_temp) & (df_forecast['destination'] == i[0])]
                df_temp_ls = df_temp['origin'].tolist()

                for a in df_temp_ls:
                    ls_a = [a]
                    ls_a.extend(i)
                    path_a.append(ls_a)

            path = path_a
            t_temp -= 1

        # 根据path列表求得每条路径对应的代价之和
        path_weight = []
        for i in path:
            weight_i = []
            for j in range(len(i) - 1):
                df_weight = df_forecast.loc[(df_forecast['origin'] == i[j]) & (df_forecast['destination'] == i[j + 1])]
                weight_ij = df_weight['weight'].tolist()[0]
                weight_i.append(1 / weight_ij)
            path_weight.append(sum(weight_i))
        # 根据path_weight列表找到代价最小的那条路径
        path_weight_min = min(path_weight)
        path_min = [path[x] for x in range(len(path)) if path_weight[x] == path_weight_min]
        return path_min, path_weight_min
    else:
        return [], []

In [5]:
# 最小生成树
# 查找根节点
def query(nodes,x):
    if nodes[x]==x:
        return x
    return query(nodes,nodes[x])

# 新增节点合并
def union(nodes,start,end):
    nodes[query(nodes,end)]=query(nodes,start)

In [ ]:
# 行业编码把字母去了
# df_graph_2024.loc[:,'source']=df_graph_2024['Source'].str[1:]
# df_graph_2024.loc[:,'target']=df_graph_2024['Target'].str[1:]
# df_graph_2024.drop(columns=['Source','Target'],inplace=True)
# df_graph_2024.rename(columns={'source':'Source','target':'Target'},inplace=True)
# df_graph_2024.loc[:,'Source']=df_graph_2024['Source'].astype('int').astype('str')
# df_graph_2024.loc[:,'Target']=df_graph_2024['Target'].astype('int').astype('str')
# 去重
# df_graph.loc[:,'source_target'] = df_graph.loc[:,['Source','Target']].apply(lambda x:sorted(x),axis=1)
# 
# df_graph.drop_duplicates(subset=['source_target']).loc[:,['Source','Target','Weight']].to_csv('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\装备制造_graph_p.csv',index=False)
# ls_nodes=list(set(df_graph['Source'].unique().tolist()+df_graph['Target'].unique().tolist()))
# df_nodes_2024_=df_nodes_2024.loc[df_nodes_2024['Id'].isin(ls_nodes)]
# df_nodes_2024_.loc[:,'tag']='out'
# df_nodes_2024_.loc[df_nodes_2024_['Id'].isin(ls_i),'tag']='inner'
# df_nodes_2024_.to_csv('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\装备制造_graph_nodes_p.csv',index=False,mode='w')

In [51]:
df_nodes_2024.loc[:,'id']=df_nodes_2024['Id'].str[1:]
df_nodes_2024.drop(columns=['Id'],inplace=True)
df_nodes_2024.rename(columns={'id':'Id'},inplace=True)
df_nodes_2024.loc[:,'Id']=df_nodes_2024['Id'].astype('int').astype('str')

In [32]:
df_graph_i_p=df_graph_2024.loc[(df_graph_2024['Source'].isin(ls_i))|(df_graph_2024['Target'].isin(ls_i))]
df_graph=pd.concat([mst,df_graph_i_p],axis=0)
df_graph

,Weight,Source,Target
0,0.750000,3421,3425
1,0.721311,3489,3591
2,0.716667,3591,4011
3,0.700000,3422,3425
4,0.695652,3563,3824
...,...,...,...
1345,0.200000,7725,3214
1360,0.142857,3593,722
1362,0.125000,3452,5293
1363,0.125000,3452,8335


In [29]:
# 根据文件切割原有图谱生成基底图谱
list_dir = os.listdir(r'E:\project_data\23shijiazhuang\graph\new_ind_graph')
for k in list_dir:
    if k.endswith('a.xlsx'):
        name=os.path.splitext(k)[0]
        name_=name.split('-')[-1][:-1]
        print(name_)
        df_i=pd.read_excel('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\'+k,dtype='str')
        ls_i=df_i['小类类别代码'].unique().tolist()
        df_graph_i=df_graph_2024_t.loc[(df_graph_2024_t['Source'].isin(ls_i))&(df_graph_2024_t['Target'].isin(ls_i))]
        df_connect=df_graph_i.loc[df_graph_i['Weight']>0]
        # df_graph_i=df_graph_2024.loc[(df_graph_2024['Source'].isin(ls_i))|(df_graph_2024['Target'].isin(ls_i))]
        
        # 用给的节点列表构成邻接矩阵，以此计算可达矩阵，判断这些点是否全连通
        # adj_matrix_chain, member_dict_chain = adjacence_matrix(ls_i, df_graph)
        mst = pd.DataFrame()
        # 首先构造一个字典,每个点的值为该店的根节点
        nodes=dict()
        ls1=df_connect['Source'].unique().tolist()
        ls2=df_connect['Target'].unique().tolist()
        ls=ls1+ls2
        ls=list(set(ls))
        for i in ls:
            nodes[i]=i
            
        # 根据联系数据，构建最小生成树
        df_relation_sk_sort=df_connect.sort_values(by=['Weight'],ascending=False).reset_index(drop=True)
        for a in range(len(df_relation_sk_sort)):
            start_i=df_relation_sk_sort.loc[a,'Source']
            end_i=df_relation_sk_sort.loc[a,'Target']
            if query(nodes,start_i) != query(nodes,end_i):
                union(nodes,start_i,end_i)
                mst=pd.concat([mst,df_relation_sk_sort.loc[[a]]])

        if name_=='生物医药':
            # 从全量联系中由大到小补充500,1000条联系
            mst_=mst.copy()
            mst_.loc[:,'Id_no']='total'
            df_relation_sk_sort=df_connect.sort_values(by=['Weight'],ascending=False).reset_index(drop=True)
            df_filt=pd.merge(left=mst_,right=df_relation_sk_sort,on=['Source','Target','Weight'],how='right')
            df_concat=df_filt.loc[df_filt['Id_no'].isnull(),['Source','Target','Weight']].head(20)
            mst_p20=pd.concat([mst,df_concat],axis=0)

            ls_nodesp20=list(set(mst_p20['Source'].unique().tolist()+mst_p20['Target'].unique().tolist()))
            df_nodes_2024_p20=df_nodes_2024.loc[df_nodes_2024['Id'].isin(ls_nodesp20)]
            print(name_,len(ls_i),len(df_connect),len(mst_p20),len(df_nodes_2024_p20))
            mst_p20.to_csv('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\'+name_+'_graph_p20_'+str(len(df_nodes_2024_p20))+'.csv',index=False)

            df_local_best=pd.read_excel(r'E:\project_data\23shijiazhuang\graph\new_ind_graph\local_best.xlsx',sheet_name=name_)
            df_nodes_new=pd.merge(left=df_nodes_2024_p20,right=df_local_best,on=['Id'],how='left')
            df_nodes_new.loc[df_nodes_new['tag'].isnull(),'tag']='none'
            df_nodes_new.loc[df_nodes_new['value'].isnull(),'value']='0'
            df_nodes_new.to_csv('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\'+name_+'_graph_nodes_p20_'+str(len(df_nodes_2024_p20))+'.csv',index=False)
        ls_nodes=list(set(mst['Source'].unique().tolist()+mst['Target'].unique().tolist()))
        df_nodes_2024_=df_nodes_2024.loc[df_nodes_2024['Id'].isin(ls_nodes)]
        print(len(ls_i),len(df_connect),len(mst),len(df_nodes_2024_))
        mst.to_csv('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\'+name_+'_graph_'+str(len(df_nodes_2024_))+'.csv',index=False)

        df_local_best=pd.read_excel(r'E:\project_data\23shijiazhuang\graph\new_ind_graph\local_best.xlsx',sheet_name=name_)
        df_nodes_new=pd.merge(left=df_nodes_2024_,right=df_local_best,on=['Id'],how='left')
        df_nodes_new.loc[df_nodes_new['tag'].isnull(),'tag']='none'
        df_nodes_new.loc[df_nodes_new['value'].isnull(),'value']='0'
        df_nodes_new.to_csv('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\'+name_+'_graph_nodes_'+str(len(df_nodes_2024_))+'.csv',index=False)
        

现代食品
150 11015 149 150
装备制造
296 41491 295 296
新材料
159 11230 158 159
现代物流
53 1361 52 53
生物医药
生物医药 67 2186 86 67
67 2186 66 67


# 图谱联系文件

In [2]:
# 专利
df_graph=pd.read_csv(r'E:\project_basedata\2024030601专利图谱\graph.csv')

In [3]:
# 数库
df_graph=pd.read_excel('E:\\project_basedata\\2023082105数库产品及联系\\t_code_product_relation_sk_0708.xlsx')
df_graph.rename(columns={'source_code':'Source','target_code':'Target','primary_weight':'Weight'},inplace=True)

In [5]:
# 国标行业图谱2021
df_graph=pd.read_csv('E:\\project_basedata\\2023082107行业图谱（2021全\\ind_connect_0705.csv',dtype={'Source':'str','Target':'str'})

In [55]:
# 国标行业图谱2024
df_graph_2024=pd.read_csv(r'E:\project_basedata\2024040301行业图谱\graph_2024.csv')
# 国标行业图谱2024_500
df_graph_2024_500=pd.read_csv(r'E:\project_basedata\2024040301行业图谱\graph_2024_p500.csv')

In [7]:
# 国标行业图谱2024_total
df_graph_2024_t=pd.read_csv(r'E:\project_basedata\2024040301行业图谱\total_connect_.csv')

# 

# 图谱节点文件

In [21]:
# 专利
df_nodes=pd.read_csv(r'E:\project_basedata\2024030601专利图谱\nodes.csv')
# 专利名称
df_nodes_info=pd.read_excel(r'E:\project_basedata\2023082103ipc专利关联数据\ipc_new.xlsx')

In [4]:
# 数库
df_nodes=pd.read_csv('E:\\project_basedata\\2023082105数库产品及联系\\nodes.csv')
# 数库节点树
df_sk=pd.read_excel('E:\\project_basedata\\2023082105数库产品及联系\\t_chains_product_20240919.xlsx',dtype='str',sheet_name='t_chains_product_tier_20240919')

In [6]:
# 国标行业
df_nodes=pd.read_csv('E:\\project_basedata\\2023082107行业图谱（2021全\\ind_nodes.csv',encoding='gbk',dtype={'Id':'str'})

In [63]:
# 国标行业全量
df_nodes_2024=pd.read_csv(r'E:\project_basedata\2024040301行业图谱\graph_2024_nodes.csv',dtype={'Id':'str'})

df_nodes_2024_500=pd.read_csv(r'E:\project_basedata\2024040301行业图谱\nodes_2024_p500.csv',dtype={'Id':'str'})

In [44]:
# 行业编码
df_ind_code=pd.read_csv(r'E:\project_basedata\2023082101国标行业\national_industry_17.csv',dtype='str')
df_ind_code_1=df_ind_code.drop_duplicates(subset=['Id_total','Label'],keep='first')

df_ind_small=df_ind_code.loc[df_ind_code['type']=='subclass']

df_ind_tier=pd.read_excel(r'\\172.30.10.237\ict\二所（数智经济研究所）\生产技术委员会\公开数据\2023082101统计局国标行业\industry_new.xlsx',sheet_name='industry_new',dtype='str')

#

# 种子文件

In [8]:
# 渭南
# 数库产品产业链图谱绘制
df_wn=pd.read_excel(r'E:\project_项目\37渭南课题\副本食品工业规上企业名单.xlsx',dtype='str')
df_wn_1=df_wn.groupby(['id','name'])['企业名称'].count().reset_index()
df_wn_1=df_wn_1.rename(columns={'id':'Id','name':'Label'})

ls_seed=df_wn_1['Id'].unique().tolist()
threshold=df_graph['Weight'].quantile(0.5)
ls_dese=[]
ls_result=simulate(ls_seed,df_graph,threshold,ls_dese)
df_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])
df_result_1=pd.merge(left=df_result,right=df_nodes.loc[:,['Id','Label']],left_on=['origin'],right_on=['Id'],how='left')
df_result_1_=df_result_1.loc[:,['t','origin','Label','destination','direction']]
df_result_1_=df_result_1_.rename(columns={'Label':'ori_Label'})
df_result_1_2=pd.merge(left=df_result_1_,right=df_nodes.loc[:,['Id','Label']],left_on=['destination'],right_on=['Id'],how='left')
df_result_1_2=df_result_1_2.rename(columns={'Label':'des_Label'})

df_graph_wn_1=df_result_1_2.loc[df_result_1_2['t']<=2]
# 根据结果反推，不能要 百货商店 CR003、综合货品商店 CR004
ls_des_result=[]
for i in range(len(df_graph_wn_1)):
    code_i=df_graph_wn_1.loc[i,'destination']
    loc_i=np.where(df_sk==code_i)
    if loc_i[0][0]>=4224:
        ls_des_result.append(code_i)
        
df_graph_wn_1_=df_graph_wn_1.loc[~(df_graph_wn_1['origin'].isin(ls_des_result))]
df_graph_wn_1_copy=df_graph_wn_1.copy()
df_graph_wn_1=df_graph_wn_1_
df_graph_wn_1.loc[df_graph_wn_1['direction']=='上游','temp']=df_graph_wn_1['origin']
df_graph_wn_1.loc[df_graph_wn_1['direction']=='上游','origin']=df_graph_wn_1['destination']
df_graph_wn_1.loc[df_graph_wn_1['direction']=='上游','destination']=df_graph_wn_1['temp']
df_graph_wn_1.loc[df_graph_wn_1['direction']=='上游','direction']='下游'
df_graph_wn_2=df_graph_wn_1.loc[:,['origin','destination','direction']]
df_graph_wn_3=pd.merge(left=df_graph_wn_2,right=df_graph,left_on=['origin','destination'],right_on=['Source','Target'],how='left')
ls_columns_=df_graph.columns.tolist()
df_graph_wn=df_graph_wn_3.loc[:,ls_columns_]

In [9]:
ls_nodes=list(set(df_graph_wn['Source'].unique().tolist()+df_graph_wn['Target'].unique().tolist()))
df_nodes_wn=df_nodes.loc[df_nodes['Id'].isin(ls_nodes)]

df_result=df_result_1_2.loc[df_result_1_2['t']<=2]
ls_step1=df_result.loc[df_result['t']==1]['destination'].unique().tolist()
df_nodes_wn.loc[df_nodes_wn['Id'].isin(ls_step1),['tag1']]='1st'
ls_step2=df_result.loc[df_result['t']==2]['destination'].unique().tolist()
df_nodes_wn.loc[df_nodes_wn['Id'].isin(ls_step2),'tag1']='2nd'
df_nodes_wn.loc[df_nodes_wn['Id'].isin(ls_seed),'tag1']='local_best'
df_nodes_wn.loc[df_nodes_wn['tag1'].isnull(),'tag1']='none'
df_nodes_wn_new=pd.merge(left=df_nodes_wn,right=df_wn_1,on=['Id','Label'],how='left')
df_nodes_wn_new.loc[df_nodes_wn_new['企业名称'].isnull(),'企业名称']=0.0

,t,origin,ori_Label,destination,direction,Id,des_Label,temp
0,1,FP00200309,啤酒,BE001,下游,FP00200309,酵母,BE001
1,1,FP001001100101,啤酒,BE001,下游,FP001001100101,麦芽,BE001
2,1,FP0010010804,啤酒,BE001,下游,FP0010010804,啤酒花,BE001
3,1,HA0010040202,啤酒,BE001,下游,HA0010040202,啤酒生产设备,BE001
4,1,HA0010040201,啤酒,BE001,下游,HA0010040201,饮料灌装机械,BE001
...,...,...,...,...,...,...,...,...
3064,2,FP00200803,畜禽养殖,CG005,下游,CG005,纺织品,NaN
3065,2,FP00200803,畜禽养殖,FP0020090207,下游,FP0020090207,其他乳制品,NaN
3066,2,FP00200803,畜禽养殖,CG0050010001,下游,CG0050010001,其他纺织品,NaN
3067,2,FP00200803,畜禽养殖,PH00100118,下游,PH00100118,动物提取物,NaN


In [13]:

df_graph_wn_1.to_csv(r'E:\project_项目\37渭南课题\gephi\数库产业链\step2.csv',index=False)
df_nodes_wn_new.to_csv(r'E:\project_项目\37渭南课题\gephi\数库产业链\nodes.csv',index=False)
df_graph_wn.to_csv(r'E:\project_项目\37渭南课题\gephi\数库产业链\graph.csv',index=False)

In [26]:
# 渭南
# 数库产品企业共现图谱绘制
# 读取规上工业企业农产品生产内容
df_wn=pd.read_excel(r'E:\project_项目\37渭南课题\副本食品工业规上企业名单.xlsx',dtype='str')
df_wn_1=df_wn.groupby(['id','name'])['企业名称'].count().reset_index()
df_wn_1=df_wn_1.rename(columns={'id':'Id','name':'Label'})

ls_seed=df_wn_1['Id'].unique().tolist()
threshold=df_graph['Weight'].quantile(0.5)
ls_dese=[]
ls_result=simulate(ls_seed,df_graph,threshold,ls_dese)
df_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])
df_result_1=pd.merge(left=df_result,right=df_nodes.loc[:,['Id','Label']],left_on=['origin'],right_on=['Id'],how='left')
df_result_1_=df_result_1.loc[:,['t','origin','Label','destination','direction']]
df_result_1_=df_result_1_.rename(columns={'Label':'ori_Label'})
df_result_1_2=pd.merge(left=df_result_1_,right=df_nodes.loc[:,['Id','Label']],left_on=['destination'],right_on=['Id'],how='left')
df_result_1_2=df_result_1_2.rename(columns={'Label':'des_Label'})

In [48]:
df_graph_wn_1=df_result_1_2.loc[df_result_1_2['t']<=2]
# 根据结果反推，不能要 百货商店 CR003、综合货品商店 CR004
ls_des_result=[]
for i in range(len(df_graph_wn_1)):
    code_i=df_graph_wn_1.loc[i,'destination']
    loc_i=np.where(df_sk==code_i)
    if loc_i[0][0]>=4224:
        ls_des_result.append(code_i)
        
df_graph_wn_1_=df_graph_wn_1.loc[~(df_graph_wn_1['origin'].isin(ls_des_result))]
df_graph_wn_1_copy=df_graph_wn_1.copy()
df_graph_wn_1=df_graph_wn_1_
df_graph_wn_1.loc[df_graph_wn_1['direction']=='上游','temp']=df_graph_wn_1['origin']
df_graph_wn_1.loc[df_graph_wn_1['direction']=='上游','origin']=df_graph_wn_1['destination']
df_graph_wn_1.loc[df_graph_wn_1['direction']=='上游','destination']=df_graph_wn_1['temp']
df_graph_wn_1.loc[df_graph_wn_1['direction']=='上游','direction']='下游'
df_graph_wn_2=df_graph_wn_1.loc[:,['origin','destination','direction']]
df_graph_wn_3=pd.merge(left=df_graph_wn_2,right=df_graph,left_on=['origin','destination'],right_on=['Source','Target'],how='left')
ls_columns_=df_graph.columns.tolist()
df_graph_wn=df_graph_wn_3.loc[:,ls_columns_]

In [49]:
ls_nodes=list(set(df_graph_wn['Source'].unique().tolist()+df_graph_wn['Target'].unique().tolist()))
df_nodes_wn=df_nodes.loc[df_nodes['Id'].isin(ls_nodes)]

df_result=df_result_1_2.loc[df_result_1_2['t']<=2]
ls_step1=df_result.loc[df_result['t']==1]['destination'].unique().tolist()
df_nodes_wn.loc[df_nodes_wn['Id'].isin(ls_step1),'tag1']='1st'
ls_step2=df_result.loc[df_result['t']==2]['destination'].unique().tolist()
df_nodes_wn.loc[df_nodes_wn['Id'].isin(ls_step2),'tag1']='2nd'
df_nodes_wn.loc[df_nodes_wn['Id'].isin(ls_seed),'tag1']='local_best'
df_nodes_wn.loc[df_nodes_wn['tag1'].isnull(),'tag1']='none'
df_nodes_wn_new=pd.merge(left=df_nodes_wn,right=df_wn_1,on=['Id','Label'],how='left')
df_nodes_wn_new.loc[df_nodes_wn_new['企业名称'].isnull(),'企业名称']=0.0

C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_20288\3656814684.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nodes_wn.loc[df_nodes_wn['Id'].isin(ls_step1),'tag1']='1st'


In [52]:
df_graph_wn_1_.to_csv(r'E:\project_项目\37渭南课题\gephi\数库企业共现产品\step2.csv',index=False)
df_nodes_wn_new.to_csv(r'E:\project_项目\37渭南课题\gephi\数库企业共现产品\nodes.csv',index=False)
df_graph_wn.to_csv(r'E:\project_项目\37渭南课题\gephi\数库企业共现产品\graph.csv',index=False)

,t,origin,ori_Label,destination,direction,Id,des_Label,temp
0,1,FP00200309,啤酒,BE001,下游,FP00200309,酵母,BE001
1,1,FP001001100101,啤酒,BE001,下游,FP001001100101,麦芽,BE001
2,1,FP0010010804,啤酒,BE001,下游,FP0010010804,啤酒花,BE001
3,1,HA0010040202,啤酒,BE001,下游,HA0010040202,啤酒生产设备,BE001
4,1,HA0010040201,啤酒,BE001,下游,HA0010040201,饮料灌装机械,BE001
...,...,...,...,...,...,...,...,...
3083,2,FA0070160101,自动化分选设备,PS0080010203,下游,PS0080010203,工业机械设备检测服务,NaN
3084,2,FA0070160101,自动化分选设备,FA00800406,下游,FA00800406,工业机械产品经销商,NaN
3085,2,FA00600906,其他畜牧业产品,FP00200814,下游,FA00600906,牧业机械及其零部件,FP00200814
3086,2,FA003002026,其他畜牧业产品,FP00200814,下游,FA003002026,牧场建设工程,FP00200814


In [70]:
k='渭南城市产业结构.xlsx'
df_i=pd.read_excel('E:\\project_项目\\37渭南课题\\'+k,sheet_name='渭南')
df_i_seed=df_i.loc[df_i['集聚度']>=1.5]
print(df_i_seed.head(5))
ls_seed_0=df_i_seed['行业代码'].unique().tolist()
df_ind_tier_i=df_ind_tier.loc[df_ind_tier['中类代码'].isin(ls_seed_0)]
len(df_ind_tier_i['小类类别代码'].unique().tolist())
df_ind_tier_i_=pd.merge(left=df_ind_tier_i,right=df_i_seed,left_on=['中类代码'],right_on=['行业代码'],how='inner')
df_ind_tier_i_1=df_ind_tier_i_.loc[:,['小类类别代码','集聚度','数量']]

            行业   行业代码   集聚度    数量
0         水果种植  A0150  6.93  3307
1  谷物、棉花等农产品仓储  G5950  6.81   108
2         其他农业  A0190  5.66  2951
3        乳制品制造  C1440  5.08    23
4         气象服务  M7410  4.32    10


In [57]:
ls_seed=df_ind_tier_i['小类类别代码'].unique().tolist()
threshold=df_graph_2024_500['Weight'].quantile(0.5)
ls_dese=[]
ls_result=simulate(ls_seed,df_graph_2024_500,threshold,ls_dese)
df_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])
df_result_1=pd.merge(left=df_result,right=df_ind_small,left_on=['origin'],right_on=['Id_total'],how='left')
df_result_1_=df_result_1.loc[:,['t','origin','Label','destination','direction']]
df_result_1_=df_result_1_.rename(columns={'Label':'ori_Label'})
df_result_1_2=pd.merge(left=df_result_1_,right=df_ind_small,left_on=['destination'],right_on=['Id_total'],how='left')
df_result_1_2=df_result_1_2.rename(columns={'Label':'des_Label'})
# print(df_result_1_2.loc[:,['t','ori_Label','des_Label','direction']].head(5))

In [66]:
df_result=df_result_1_2.loc[df_result_1_2['t']<=2]
ls_step1=df_result.loc[df_result['t']==1]['destination'].unique().tolist()
df_nodes_2024_500.loc[df_nodes_2024_500['Id'].isin(ls_step1),'tag1']='1st'
ls_step2=df_result.loc[df_result['t']==2]['destination'].unique().tolist()
df_nodes_2024_500.loc[df_nodes_2024_500['Id'].isin(ls_step2),'tag1']='2nd'
df_nodes_2024_500.loc[df_nodes_2024_500['Id'].isin(ls_seed),'tag1']='local_best'
df_nodes_2024_500.loc[df_nodes_2024_500['tag1'].isnull(),'tag1']='none'
# df_nodes_2024_500.to_csv(r'E:\project_项目\37渭南课题\weinan.csv',index=False)

In [73]:
df_nodes_2024_500_=pd.merge(left=df_nodes_2024_500,right=df_ind_tier_i_1,left_on=['Id'],right_on=['小类类别代码'],how='left')
df_nodes_2024_500_.fillna(0.0).to_csv(r'E:\project_项目\37渭南课题\gephi\weinan.csv',index=False)

In [42]:
k='product.xlsx'
df_i=pd.read_excel('E:\\project\\37product\\'+k,sheet_name='pro_total')
df_i_seed=df_i.loc[df_i['集聚度']>=1.5]
print(df_i_seed.loc[:,['product','集聚度','数量']].head(5))

            行业   集聚度    数量
0         水果种植  6.93  3307
1  谷物、棉花等农产品仓储  6.81   108
2         其他农业  5.66  2951
3        乳制品制造  5.08    23
4         气象服务  4.32    10


In [39]:
ls_seed=df_i_seed['product'].unique().tolist()
threshold=df_graph_2024['Weight'].quantile(0.5)
ls_dese=[]
ls_result=simulate(ls_seed,df_graph_2024,threshold,ls_dese)
df_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])
df_result_1=pd.merge(left=df_result,right=df_ind_small,left_on=['origin'],right_on=['Id_total'],how='left')
df_result_1_=df_result_1.loc[:,['t','origin','Label','destination','direction']]
df_result_1_=df_result_1_.rename(columns={'Label':'ori_Label'})
df_result_1_2=pd.merge(left=df_result_1_,right=df_ind_small,left_on=['destination'],right_on=['Id_total'],how='left')
df_result_1_2=df_result_1_2.rename(columns={'Label':'des_Label'})
print(df_result_1_2.loc[:,['t','ori_Label','des_Label','direction']].head(5))

   t ori_Label     des_Label direction
0  1      其他农业      其他未列明畜牧业        下游
1  1      其他农业  其他林业专业及辅助性活动        下游
2  1      其他农业        其他公路客运        下游
3  2  其他未列明畜牧业          猪的饲养        上游
4  2  其他未列明畜牧业         食用菌种植        上游


In [43]:
df_result=df_result_1_2.loc[df_result_1_2['t']<=2]
ls_step1=df_result.loc[df_result['t']==1]['destination'].unique().tolist()
df_nodes_2024.loc[df_nodes_2024['Id'].isin(ls_step1),'tag1']='1st'
ls_step2=df_result.loc[df_result['t']==2]['destination'].unique().tolist()
df_nodes_2024.loc[df_nodes_2024['Id'].isin(ls_step2),'tag1']='2nd'
df_nodes_2024.loc[df_nodes_2024['Id'].isin(ls_seed),'tag1']='local_best'
df_nodes_2024.loc[df_nodes_2024['tag1'].isnull(),'tag1']='none'
print(df_nodes_2024.loc[:,['Label','tag1']].head(5))

         Label  tag1
0      漆器工艺品制造  none
1       海洋石油开采  none
2    其他稀有金属矿采选  none
3  其他文体设备和用品出租  none
4      雕塑工艺品制造  none


In [38]:
len(df_ind_code_1.loc[df_ind_code_1['Id_total'].isin(ls_seed)])

28

In [28]:
df_nodes_2024.to_csv(r'E:\project_项目\37渭南课题\weinan.csv',index=False)

In [17]:
k='国标总表-sjz-生物医药a.xlsx'
df_i=pd.read_excel('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\'+k,dtype='str')
ls_seed=df_i['小类类别代码'].unique().tolist()
threshold=df_graph_2024['Weight'].quantile(0.5)
ls_dese=[]
ls_result=simulate(ls_seed,df_graph_2024,threshold,ls_dese)
df_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])
df_result_1=pd.merge(left=df_result,right=df_ind_small,left_on=['origin'],right_on=['Id_total'],how='left')
df_result_1_=df_result_1.loc[:,['t','origin','Label','destination','direction']]
df_result_1_=df_result_1_.rename(columns={'Label':'ori_Label'})
df_result_1_2=pd.merge(left=df_result_1_,right=df_ind_small,left_on=['destination'],right_on=['Id_total'],how='left')
df_result_1_2=df_result_1_2.rename(columns={'Label':'des_Label'})
df_result_1_2.loc[:,['t','origin','ori_Label','destination','des_Label','direction']].to_csv(r'E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\生物医药补充.csv',index=False)

### 石家庄经开区五个产业方向模拟

In [30]:
ls_dire=['生物医药','新材料','现代物流','现代食品']
ls_num=['67','159','53','150']
for i in range(len(ls_dire)):
    name_=ls_dire[i]
    df_local_best=pd.read_excel(r'E:\project_data\23shijiazhuang\graph\new_ind_graph\local_best.xlsx',sheet_name=name_)
    ls_seed=df_local_best['Id'].unique().tolist()
    if name_=='生物医药':
        df_graph=pd.read_csv('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\'+name_+'_graph_p20_'+ls_num[i]+'.csv')
    else:
        df_graph=pd.read_csv('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\'+name_+'_graph_'+ls_num[i]+'.csv')
    threshold=df_graph['Weight'].quantile(0.5)
    ls_dese=[]
    ls_result=simulate(ls_seed,df_graph,threshold,ls_dese)
    df_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])
    df_nodes=pd.read_csv('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\'+name_+'_graph_nodes_'+ls_num[i]+'.csv')
    ls_step1=df_result.loc[df_result['t']==1]['destination'].unique().tolist()
    df_nodes.loc[df_nodes['Id'].isin(ls_step1),'tag1']='1st'
    ls_step2=df_result.loc[df_result['t']==2]['destination'].unique().tolist()
    df_nodes.loc[df_nodes['Id'].isin(ls_step2),'tag1']='2nd'
    df_nodes.loc[df_nodes['Id'].isin(ls_seed),'tag1']='local_best'
    df_nodes.loc[df_nodes['tag1'].isnull(),'tag1']='none'
    df_nodes.to_csv('E:\\project_data\\23shijiazhuang\\graph\\new_ind_graph\\'+name_+'_graph_nodes_'+ls_num[i]+'_tag1.csv',index=False,mode='w')

In [34]:
name_='装备制造'
df_local_best=pd.read_excel(r'E:\project_data\23shijiazhuang\graph\new_ind_graph\local_best.xlsx',sheet_name=name_)
ls_seed=df_local_best['Id'].unique().tolist()
df_graph=pd.read_csv(r'E:\project_data\23shijiazhuang\graph\new_ind_graph\装备制造_graph_296.csv')
threshold=df_graph['Weight'].quantile(0.5)
ls_dese=[]
ls_result=simulate(ls_seed,df_graph,threshold,ls_dese)
df_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])

In [38]:
df_nodes=pd.read_csv(r'E:\project_data\23shijiazhuang\graph\new_ind_graph\装备制造_graph_nodes_296.csv')

ls_step1=df_result.loc[df_result['t']==1]['destination'].unique().tolist()
df_nodes.loc[df_nodes['Id'].isin(ls_step1),'tag1']='1st'
ls_step2=df_result.loc[df_result['t']==2]['destination'].unique().tolist()
df_nodes.loc[df_nodes['Id'].isin(ls_step2),'tag1']='2nd'
df_nodes.loc[df_nodes['Id'].isin(ls_seed),'tag1']='local_best'
df_nodes.loc[df_nodes['tag1'].isnull(),'tag1']='none'
df_nodes.to_csv(r'E:\project_data\23shijiazhuang\graph\new_ind_graph\装备制造_graph_nodes_296.csv',index=False,mode='w')

### 石家庄经开区农业模拟

In [17]:

df_shijiazhuang_1chan=pd.read_csv(r'E:\project_data\23shijiazhuang\graph\ind_graph\country_divide_gcq_ind.csv')
df_shijiazhuang_1chan_=df_shijiazhuang_1chan.loc[(df_shijiazhuang_1chan['type1']=='农、林、牧、渔业')&(df_shijiazhuang_1chan['tag']=='local_best')]
ls_seed=[str(x) for x in df_shijiazhuang_1chan_['Id'].tolist()]
threshold=df_graph['Weight'].quantile(0.5)
ls_dese=[]
ls_result=simulate(ls_seed,df_graph,threshold,ls_dese)
df_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])

In [22]:
ls_step1=df_result.loc[df_result['t']==1]['destination'].unique().tolist()
df_nodes.loc[df_nodes['Id'].isin(ls_step1),'tag']='1st'
ls_step2=df_result.loc[df_result['t']==2]['destination'].unique().tolist()
df_nodes.loc[df_nodes['Id'].isin(ls_step2),'tag']='2nd'
ls_step3=df_result.loc[df_result['t']==3]['destination'].unique().tolist()
df_nodes.loc[df_nodes['Id'].isin(ls_step3),'tag']='3nd'
ls_step4=df_result.loc[df_result['t']==4]['destination'].unique().tolist()
df_nodes.loc[df_nodes['Id'].isin(ls_step4),'tag']='4th'
ls_step5=df_result.loc[df_result['t']==5]['destination'].unique().tolist()
df_nodes.loc[df_nodes['Id'].isin(ls_step5),'tag']='5th'

df_nodes.loc[df_nodes['Id'].isin(ls_seed),'tag']='local_best'
df_nodes.loc[df_nodes['tag'].isnull(),'tag']='none'

# df_nodes_=pd.merge(left=df_nodes,right=df_seed,on=['Id'],how='left')
# df_nodes_=pd.merge(left=df_nodes_,right=df_nodes_info,left_on=['大类'],right_on=['code'],how='left')
# df_nodes_.drop(labels=['code'],axis=1,inplace=True)
# df_nodes_.rename(columns={'name':'Dname'},inplace=True)
# 
# df_nodes_.to_csv('E:\\project_data\\23shijiazhuang\\graph\\patent\\sjz_nodes.csv',index=False)

In [24]:
df_nodes.to_csv(r'E:\project_data\23shijiazhuang\graph\ind_graph\1chan_nodes.csv',index=False)

### 石家庄经开区农业模拟结束

In [27]:
df_nodes_=pd.merge(left=df_nodes_,right=df_nodes_info,left_on=['大类'],right_on=['code'],how='left')
df_nodes_.drop(labels=['code'],axis=1,inplace=True)
df_nodes_.rename(columns={'name':'Dname'},inplace=True)
df_nodes_

KeyError: "['code'] not found in axis"

In [29]:
# 专利
df_seed=pd.read_excel(r'E:\project_data\23shijiazhuang\graph\patent\石家庄藁城区优势专利.xlsx')
df_seed.rename(columns={'IPC分类号':'Id','专利数量':'num'},inplace=True)
df_seed=df_seed.loc[:,['Id','num']]
ls_seed=df_seed['Id'].tolist()
threshold=df_graph['Weight'].quantile(0.5)
ls_dese=[]
ls_result=simulate(ls_seed,df_graph,threshold,ls_dese)
df_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])

ls_step1=df_result.loc[df_result['t']==1]['destination'].unique().tolist()
df_nodes.loc[df_nodes['Id'].isin(ls_step1),'tag']='1st'
ls_step2=df_result.loc[df_result['t']==2]['destination'].unique().tolist()
df_nodes.loc[df_nodes['Id'].isin(ls_step2),'tag']='2nd'
df_nodes.loc[df_nodes['Id'].isin(ls_seed),'tag']='local_best'
df_nodes.loc[df_nodes['tag'].isnull(),'tag']='none'

df_nodes_=pd.merge(left=df_nodes,right=df_seed,on=['Id'],how='left')
df_nodes_=pd.merge(left=df_nodes_,right=df_nodes_info,left_on=['大类'],right_on=['code'],how='left')
df_nodes_.drop(labels=['code'],axis=1,inplace=True)
df_nodes_.rename(columns={'name':'Dname'},inplace=True)

df_nodes_.to_csv('E:\\project_data\\23shijiazhuang\\graph\\patent\\sjz_nodes.csv',index=False)

In [5]:
# 数库
df_seed=pd.DataFrame()
df_seed_1=pd.read_excel('E:\\project_data\\23shijiazhuang\\seed\\筛选产品-装备制造产业方向1227.xlsx',sheet_name='Sheet1')
ls_seed_1=df_seed_1.loc[df_seed_1['是否汇总']=='是']['产品编码'].unique().tolist()
df_seed_1.rename(columns={'产品编码':'Id','产品名称':'Label'},inplace=True)
ls_columns_num=[x for x in range(len(df_sk.columns))]
ls_columns=df_sk.columns.tolist()
for i in ls_seed_1:
    tag_i=df_seed_1.loc[df_seed_1['Id']==i]['现状/未来发展'].unique().tolist()[0]
    
    loc_i=np.where(df_sk==i)
    num_i=list(set(loc_i[1]))[0]
    df_i=df_sk.iloc[loc_i[0],ls_columns_num[num_i+2:]]
    for j in range(int(df_sk.columns.tolist()[num_i+2:][0][-1]),10):
        code_j='code_'+str(j)
        name_j='name_'+str(j)
        df_j=df_i.loc[:,[code_j,name_j]].drop_duplicates()
        df_j.rename(columns={code_j:'Id',name_j:'Label'},inplace=True)
        df_j.loc[:,'现状/未来发展']=tag_i
        df_seed=pd.concat([df_seed,df_j])
df_seed=df_seed.drop_duplicates()
df_seed_2=df_seed.loc[df_seed['Id']!='0']
df_seed_3=pd.concat([df_seed_1,df_seed_2],axis=0)
df_seed_3=df_seed_3.drop_duplicates()

In [35]:
#
df_center=pd.read_csv(r'E:\project_data\25issue\center.csv')
df_yuanqu=pd.read_csv(r'E:\project_data\25issue\yuanqu.csv')
df_center_=df_center.copy()
df_center_.loc[:,'city']=df_center_['name'].str[:3]
df_center_.rename(columns={'x':'x_o','y':'y_o'},inplace=True)
df_center_1=df_center_.loc[:,['city','x_o','y_o']]
df_yuanqu.rename(columns={'x':'x_d','y':'y_d'},inplace=True)
df_result=pd.merge(left=df_yuanqu,right=df_center_1,on=['city'],how='left')
df_result.to_csv(r'E:\project_data\25issue\xy_line.csv',index=False)

In [33]:
df_center_=df_center.copy()
df_center_.loc[:,'city']=df_center_['name'].str[:3]
df_center_

,FID,Join_Count,UID,name,province,city,district,mu,gd_loc,url,面积,园区类,ORIG_FID,x,y
0,0,0,,北京市中心,,北京市,,,,,0.0,,0,116.397618,39.908272
1,1,0,,上海市中心,,上海市,,,,,0.0,,0,121.501689,31.238588
2,2,0,,广州市中心,,广州市,,,,,0.0,,0,113.324501,23.106390
3,3,0,,深圳市中心,,深圳市,,,,,0.0,,0,114.059715,22.541635


In [39]:
# 国标行业
# 模拟
df_seed_3=pd.read_excel('E:\\project_data\\23shijiazhuang\\藁城区行业以石家庄为分母rca.xlsx')

df_seed_3.loc[:,'ind_Id']=df_seed_3['行业代码'].str[1:]
df_seed_3=df_seed_3.astype(dtype={'ind_Id':'int'})
df_seed_3=df_seed_3.astype(dtype={'ind_Id':'str'})
df_seed_3_=df_seed_3.loc[:,['ind_Id','rca']]
df_seed_3_.rename(columns={'ind_Id':'Id'},inplace=True)

ls_ind_best=df_seed_3.loc[df_seed_3['rca']>=3]['行业代码'].unique().tolist()
code_ls=[str(int(x[1:])) for x in ls_ind_best]
df_connect=df_graph
threshold=df_connect['Weight'].quantile(0.3)
ls_dese=[]
ls_result=simulate(code_ls,df_connect,threshold,ls_dese)
df_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])

# 绘图
# df_result_2=df_result.loc[df_result['t']<=2]
# ls_result_node1=df_result_2['origin'].unique().tolist()
# ls_result_node2=df_result_2['destination'].unique().tolist()
# ls_result_node=list(set(ls_result_node1+ls_result_node2))
# df_nodes_result=df_nodes.loc[df_nodes['Id'].isin(ls_result_node)]

ls_step1=df_result.loc[df_result['t']==1]['destination'].unique().tolist()
df_nodes.loc[df_nodes['Id'].isin(ls_step1),'tag']='1st'
ls_step2=df_result.loc[df_result['t']==2]['destination'].unique().tolist()
df_nodes.loc[df_nodes['Id'].isin(ls_step2),'tag']='2nd'
df_nodes.loc[df_nodes['Id'].isin(code_ls),'tag']='local_best'
df_nodes.loc[df_nodes['tag'].isnull(),'tag']='none'

# df_graph_result=df_graph.loc[(df_graph['Source'].isin(ls_result_node))&(df_graph['Target'].isin(ls_result_node))&(df_graph['Weight']>=threshold)]
# df_graph_result.to_csv('E:\\project_data\\23shijiazhuang\\graph\\shipin_graph_2step_t41.csv',index=False)
df_nodes_=pd.merge(left=df_nodes,right=df_seed_3_,on=['Id'],how='left')
df_nodes_.loc[df_nodes_['rca'].isnull(),'rca']=0
df_nodes_.to_csv('E:\\project_data\\23shijiazhuang\\graph\\ind_graph\\shijiazhuang_divide_gcq_ind.csv',index=False)

In [7]:
# 数库模拟
# 医药避免
# ls_dese=['PS00500310','PS0080010203','FA007031','PS008010','FN005002','FA00800406','PS0050042401','MM018001','MM0180010302']
# 物流避免
# ls_dese=['FA007031','MM018001','PS008010','PS00500310','PS0050042401','PS0080010203','FA00800406','TR00100515','BT01004','EC003003','TR00100512','TR00100509','TR0010050403']
ls_dese=[]
code_ls1=df_seed_3['Id'].unique().tolist()
code_ls=[x for x in code_ls1 if x not in ls_dese]
df_connect=df_graph
threshold = 4
# df_connect['Weight'].quantile(0.1)
ls_result=simulate(code_ls,df_connect,threshold,ls_dese)
df_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])

In [18]:
df_result.to_csv('E:\\project_data\\23shijiazhuang\\graph\\cailiao_step_all2.csv',index=False,mode='w')

In [9]:
# 绘图
# 物流终点
# ls_end=['TR00100504','FN00500202','FN00500201','FN005002']
ls_end=[]
df_seed_3_=df_seed_3.loc[:,['Id','Label','现状/未来发展']]
df_result_2=df_result.loc[(df_result['t']<=1)&(~(df_result['origin'].isin(ls_end)))]
ls_result_node1=df_result_2['origin'].unique().tolist()
ls_result_node2=df_result_2['destination'].unique().tolist()
ls_result_node=list(set(ls_result_node1+ls_result_node2))
df_nodes_result=df_nodes.loc[df_nodes['Id'].isin(ls_result_node)]

df_nodes_result_1=pd.merge(left=df_nodes_result,right=df_seed_3_,on=['Id','Label'],how='left')
df_nodes_result_1.loc[df_nodes_result_1['Id'].isin(code_ls),'type_name']=df_nodes_result_1['现状/未来发展']

df_graph_result=df_graph.loc[(df_graph['Source'].isin(ls_result_node))&(df_graph['Target'].isin(ls_result_node))&(df_graph['Weight']>=threshold)]
df_graph_result.to_csv('E:\\project_data\\23shijiazhuang\\graph\\zhuangbei_graph_1step_t4.csv',index=False)
df_nodes_result_1.to_csv('E:\\project_data\\23shijiazhuang\\graph\\zhuangbei_nodes_1step_t4.csv',index=False)

In [16]:
df_result

,t,origin,destination,direction
0,1,MM018001,FA007034,上游
1,1,MM018001,MM0180081303,上游
2,1,MM018001,MM0180081401,上游
3,1,MM018001,FA00703402,上游
4,1,MM018001,FA00703401,上游
...,...,...,...,...
16653,5,FN002001,PS005029,上游
16654,5,IT001008,SS00101109,下游
16655,5,FN002,PS00800303,上游
16656,5,FN002,PS005029,上游


In [116]:
df_seed_3.loc[df_seed_3['现状/未来发展']=='现状优势']

,方向名称,现状/未来发展,Id,Label,是否汇总
2,生物医药,现状优势,PH002008010201,输液产品,NaN
5,生物医药,现状优势,HC00200105,输液器,NaN
6,生物医药,现状优势,PH002008010201,输液产品,NaN
8,生物医药,现状优势,PH002011,药用辅料,NaN
2814,NaN,现状优势,PH0020010310,注射用夫西地酸钠,NaN
2815,NaN,现状优势,PH0020010313,青霉素类,NaN
2819,NaN,现状优势,PH0020010315,诺氟沙星,NaN
2820,NaN,现状优势,PH0020010316,盐酸芦氟沙星,NaN
2821,NaN,现状优势,PH0020010317,阿奇霉素,NaN
2822,NaN,现状优势,PH0020010320,乙酰螺旋霉素,NaN


In [113]:
df_nodes

,Id,Label,level,type_code,type_name
0,CC0010092407,塑料人造皮革,4,CSF_151010,化学制品
1,SE00203106,打印机耗材芯片,6,CSF_453010,半导体产品与设备
2,CC00500308,其他胶粘剂,3,CSF_151010,化学制品
3,HW001012,移动通信设备,4,CSF_452010,通信设备
4,EG001007,其他油气相关服务,3,CSF_101010,能源设备与服务
...,...,...,...,...,...
4921,FA00706634,电子柜锁,4,CSF_201060,机械制造
4922,UT00400405,大型水处理系统集成,5,CSF_202010,商业服务与商业用品
4923,CC001007031012,六氟丙烯（HFP）,3,CSF_151010,化学制品
4924,CC001007030303,碳酸锂,3,CSF_151010,化学制品


In [70]:
df_nodes
for i in range(len(df_nodes)):
    node_i=df_nodes.loc[i,'Id']
    loc_i=np.where(df_sk==node_i)[0][0]
    df_nodes.loc[i,'type_code']=df_sk.loc[loc_i,'code_0']
    df_nodes.loc[i,'type_name']=df_sk.loc[loc_i,'name_0']

In [72]:
df_nodes.to_csv('E:\\project_basedata\\2023082105数库产品及联系\\nodes.csv',index=False)

In [55]:
df_seed

,Id,Label
2352,FP0010010102,谷类
2358,FP0010010103,豆类
2361,FP0010010104,油料作物
2363,FP0010010105,薯类
2365,FP0010010106,其他粮油作物
...,...,...
2376,0,0
2374,0,0
2374,0,0
2374,0,0
